# LSTMによるレストランの客数予測

### 目標
821店舗のレストランにおける2017年4月最終週〜2017年5月末日までの各日にちに来店したお客さんの数を過去の予約や実客数のデータから予測  
$\Rightarrow$ 821店舗×39日間の実客数を予測
### データの内容
* **air_reserve.csv – Airレジ経由の予約情報**
 * air_store_id – AirレジのレストランID
 * visit_datetime – 予約時の訪問予定時間
 * reserve_datetime – 予約した時の日付時刻
 * reserve_visitors – 予約人数


* **air_store_info.csv – Airレジのレストラン情報**
 * air_store_id – AirレジのレストランID
 * air_genre_name – レストランのジャンル
 * air_area_name – レストランの場所
 * latitude – 緯度
 * longitude – 経度


* **air_visit_data.csv – Airレジの各レストランの日付ごとの実客数**
 * air_store_id – AirレジのレストランID
 * visit_date – 日付
 * visitors – 実客数


* **data_info.csv – 日付の基本的な情報**


* **hpg_reserve.csv – ホットペッパー経由の予約情報**
 * hpg_store_id – ホットペッパーのレストランID
 * visit_datetime – 予約時の訪問予定時間
 * reserve_datetime – 予約した時の日付時刻
 * reserve_visitors – 予約人数


* **hpg_store_info.csv – ホットペッパーのレストラン情報**
 * hpg_store_id – ホットペッパーのレストランID
 * hpg_genre_name – レストランのジャンル
 * hpg_area_name – レストランの場所
 * latitude – 緯度
 * longitude – 経度


* **sample_submission.csv – 提出ファイルのサンプル**
 * id – air_store_idとvisit_dateを連結させたID
 * visitors – 予想客数


* **store_id_relation.csv – AirレジとホットペッパーのIDの紐付け（両サービスを使っているお店のみ）**
 * hpg_store_id – ホットペッパーのレストランID
 * air_store_id – AirレジのレストランID

In [ ]:
# 基本的なライブラリの読み込み
## データ取得,整形,可視化などのライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. データの取得＆調整
## 1.1 データの取得＆欠損値の確認

1. **/kaggle/input/**下のファイルからデータを取得し，DataFrameに格納
1. `isnull関数`と`sum関数`より欠損値をカウント

In [ ]:
# Airレジ経由の予約情報の取得
air_reserve = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_reserve.csv.zip', parse_dates=['visit_datetime', 'reserve_datetime'])
air_reserve.head()

In [ ]:
# ホットペッパー経由の予約情報の取得
hpg_reserve = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/hpg_reserve.csv.zip', parse_dates=['visit_datetime', 'reserve_datetime'])
hpg_reserve.head()

In [ ]:
# AirレジとホットペッパーのIDの紐付け
air_hpg_mapping =  pd.read_csv('../input/recruit-restaurant-visitor-forecasting/store_id_relation.csv.zip')
air_hpg_mapping.head()

In [ ]:
# Airレジのレストラン情報
air_info = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_store_info.csv.zip')
air_info.head()

In [ ]:
# ホットペッパーのレストラン情報
hpg_info = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/hpg_store_info.csv.zip')
hpg_info.head()

In [ ]:
# Airレジの各レストランの日付ごとの実客数
visit_data = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_visit_data.csv.zip', parse_dates=['visit_date'])
visit_data['visit_date'] = visit_data.visit_date.astype(str)
# 辞書型に変換
visit_data_dict = visit_data.set_index(['air_store_id', 'visit_date'])['visitors'].to_dict()
visit_data.head()

In [ ]:
# 欠損値の数を確認
print('*** air_reserve ***\n', air_reserve.isnull().sum())
print('*** hpg_reserve ***\n', hpg_reserve.isnull().sum())
print('*** air_hpg_mapping ***\n', air_hpg_mapping.isnull().sum())
print('*** air_info ***\n', air_info.isnull().sum())
print('*** hpg_info ***\n', hpg_info.isnull().sum())
print('*** visit_data ***\n', visit_data.isnull().sum())

## 1.2 訓練データの調整

テストデータと訓練データの差異を調査し，差が埋まるように訓練データの調整を行う．

In [ ]:
# 目標： レストランIDごとのvisitorsの予測
submission_data = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/sample_submission.csv.zip')
submission_data.head()

In [ ]:
submission_data['air_store_id'] = submission_data.id.apply(lambda x: '_'.join(x.split('_')[:-1]))
submission_data['visit_date'] = pd.to_datetime(submission_data.id.apply(lambda x: x.split('_')[-1]))
submission_data.head()

In [ ]:
holiday_data = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/date_info.csv.zip',parse_dates=['calendar_date'])
holiday_data.head()

In [ ]:
#　submission_data(テストデータ)に含まれるすべてのレストランIDがvisit_data(訓練データ)に含まれているかどうかを確認
test_air_ids = submission_data.air_store_id.unique()
train_air_ids = visit_data.air_store_id.unique()

print('テストデータ数：', len(test_air_ids),'訓練データ数：', len(train_air_ids))

In [ ]:
# visit_data(訓練データ)の中でsubmission_data(テストデータ)に含まれていないレストランID
set(train_air_ids) - set(test_air_ids)

In [ ]:
#Step 1: テスト用レストランIDを数値化して保存
air_id_dict = dict([(*zip(test_air_ids, np.arange(len(test_air_ids))))])
air_id_reverse_dict = dict([*zip(air_id_dict.values(), air_id_dict.keys())])

#Step 2: レストランIDで訓練データを初期化
train_data = pd.DataFrame({'air_store_id': test_air_ids})
print('訓練データ数：', len(train_data))
train_data.head()

In [ ]:
#Step 3: 訓練データの各データについて、日付を取得し、訪問者をマッピングする
#訓練データ ： 2016-01-01 〜 2017-04-22 (reserve)
date_cols = pd.date_range(start='2016-01-01', end='2017-04-22').date.astype(str)

for col in date_cols:
    train_data[col] = train_data.air_store_id.apply(lambda x: visit_data_dict[(x, col)] 
                                                    if (x, col) in visit_data_dict.keys()
                                                    else -1).astype(np.int32)
print('訓練データの行数と列数：', train_data.shape)
train_data.head()

In [ ]:
# 訓練データにレストランの情報を追加
train_data = pd.merge(train_data, air_info, on='air_store_id', how='left')
print('訓練データの行数と列数：', train_data.shape)
train_data.head()

In [ ]:
train_data.air_area_name.nunique(),train_data.air_genre_name.nunique()

In [ ]:
plt.scatter(train_data.latitude, train_data.longitude)
plt.show()

In [ ]:
train_data.latitude.nunique(), train_data.longitude.nunique()

In [ ]:
air_reserve.tail()

In [ ]:
len(pd.date_range("2017-04-23", "2017-05-31")) #visit datetime

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
train_data['air_genre_name'] = lb.fit_transform(train_data['air_genre_name'])
train_data['air_area_name']  = lb.fit_transform(train_data['air_area_name'])
train_data.head()

# 2. 正規化
`MinMaxScaler`を使って`train data` の最大値を1, 最小値を0として正規化する．

In [ ]:
def log_transform(data):
    data = data.applymap(lambda x: 0 if x==-1 else x)
    data.iloc[:, 1:479] = np.log1p(data.iloc[:, 1:479])
    return data

TRdata = log_transform(train_data)
TRdata.head(10)
TRdata.head(10)

# 3. LSTMの入力を生成

In [ ]:
#
from sklearn.model_selection import train_test_split

def generate_walkforward_data(data, input_seq_len, output_seq_len, date_start_col, date_end_col, val_data=0.05):
    date_cols = np.r_[date_start_col: date_end_col]
    train_points = len(date_cols) - input_seq_len - output_seq_len
    input_seqs = []
    output_seqs = []
    for i in range(train_points+1):
        inp_start = i + date_start_col
        inp_end = inp_start + input_seq_len
        out_end = inp_end + output_seq_len 
        input_seqs.append( data.iloc[: , inp_start:inp_end].values.reshape(1, -1, input_seq_len).transpose(0, 2, 1) )
        output_seqs.append( data.iloc[: , inp_end:out_end].values.reshape(1, -1, output_seq_len).transpose(0, 2, 1) )
        
    input_seqs = np.concatenate(input_seqs)
    output_seqs = np.concatenate(output_seqs)
    
    train_x, val_x, train_y, val_y = train_test_split(input_seqs, output_seqs, test_size=val_data, random_state=1126)
    
    return train_x, val_x, train_y, val_y

In [ ]:
input_seq_len = 39
output_seq_len = 39
train_x, val_x, train_y, val_y = generate_walkforward_data(TRdata, input_seq_len, output_seq_len, 1, 479)
print(train_x.shape, train_y.shape, val_x.shape, val_y.shape)

# 4. モデルの作成

In [ ]:
#  モデル作成用ライブラリの読み込み
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [ ]:
def buildManyToManyModel(train_data):
    model = Sequential()
    model.add(LSTM(10, input_shape = (train_data.shape[1], train_data.shape[2]),return_sequences=True))
    model.add(TimeDistributed(Dense(821)))
    model.compile(loss="mse", optimizer="adam")
    return model

In [ ]:
model = buildManyToManyModel(train_x)
model.summary()

# 5. 学習

In [ ]:
callback = EarlyStopping(monitor="loss", patience=3, verbose=1, mode="auto")
history = model.fit(train_x, train_y, epochs=1000, batch_size=50, validation_data=(val_x, val_y), callbacks=[callback])

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

In [ ]:
test_output = model.predict(train_x)
test_output.shape

In [ ]:
train_points = 478 - 39 - 39
date_start_col = 1
inp_start = train_points + date_start_col
inp_end = inp_start + input_seq_len
out_end = inp_end + output_seq_len 
last_train_day = TRdata.iloc[: , inp_end:out_end]
last_train_day

In [ ]:
last_train_day = last_train_day.values.reshape(1, -1, output_seq_len).transpose(0, 2, 1)
last_train_day.shape

In [ ]:
last_train_day[0]

# 6. 学習結果より客数を予測

In [ ]:
test_output = model.predict(last_train_day)
test_output.shape

In [ ]:
test_output

In [ ]:
store = np.exp(test_output[0]) - 1
store[:,6]

In [ ]:
people = store[:,6]
day = np.arange(1,40)
plt.plot(day,people)

# 7. 提出ファイルに書き出し

In [ ]:
visitor = np.exp(test_output[0]) - 1
visitor.shape

In [ ]:
result = visitor.T.ravel()
result.shape

In [ ]:
datetime = pd.date_range('2017-04-23',periods=39)
string_list = []
for i in range(len(TRdata['air_store_id'])): #821
    for j in range(len(datetime)): #39
        string_list.append(TRdata['air_store_id'][i]+'_'+datetime[j].strftime('%Y-%m-%d'))
        
df = pd.DataFrame(result, index = string_list,columns=['visitors'])
df.index.name = 'id'
df

In [ ]:
df.to_csv('result2.csv')